In [3]:
# Step 2: Import Libraries and Set Up
import torch
import cv2
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline

# Debug: Print the current working directory
print("Current working directory:", os.getcwd())

# Ensure output directory exists (one level up in the root directory)
os.makedirs("../data/output", exist_ok=True)

Current working directory: /home/shreyx/Desktop/Projects/NNDL Mini Project/Automated-Seat-allocation/src


In [4]:
# Step 3: Define Chair Detection Function
def detect_chairs(image_path, iou_threshold=0.5):
    # Load the YOLOv5 model
    model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

    # Load the classroom image
    img = cv2.imread(image_path)
    if img is None:
        raise FileNotFoundError(f"Could not load image at {image_path}. Check the file path and ensure the image exists.")

    # Perform inference
    results = model(img)

    # Get the detected objects
    detections = results.xyxy[0].numpy()

    # Filter for chairs (class 56) and people (class 0)
    chair_detections = detections[detections[:, 5] == 56]
    person_detections = detections[detections[:, 5] == 0]

    # Assign unique identifiers to each chair
    chairs = []
    for i, detection in enumerate(chair_detections):
        x1, y1, x2, y2, conf, _ = detection
        center_x = (x1 + x2) / 2
        center_y = (y1 + y2) / 2
        chairs.append({
            "x1": x1,
            "y1": y1,
            "x2": x2,
            "y2": y2,
            "center_x": center_x,
            "center_y": center_y
        })

    chairs_df = pd.DataFrame(chairs)

    # Sort chairs by center_y (rows) and then center_x (within rows)
    chairs_df = chairs_df.sort_values(by=["center_y", "center_x"]).reset_index(drop=True)

    # Assign Seat IDs after sorting
    chairs_df["Seat ID"] = [f"Seat {i+1}" for i in range(len(chairs_df))]

    # Determine occupancy status
    def is_occupied(chair, person_detections, iou_threshold):
        chair_box = [chair["x1"], chair["y1"], chair["x2"], chair["y2"]]
        for person in person_detections:
            person_box = person[:4]
            x1 = max(chair_box[0], person_box[0])
            y1 = max(chair_box[1], person_box[1])
            x2 = min(chair_box[2], person_box[2])
            y2 = min(chair_box[3], person_box[3])
            intersection = max(0, x2 - x1) * max(0, y2 - y1)
            chair_area = (chair_box[2] - chair_box[0]) * (chair_box[3] - chair_box[1])
            person_area = (person_box[2] - person_box[0]) * (person_box[3] - person_box[1])
            union = chair_area + person_area - intersection
            iou = intersection / union if union > 0 else 0
            if iou > iou_threshold:
                return True
        return False

    chairs_df["Status"] = chairs_df.apply(lambda row: "Occupied" if is_occupied(row, person_detections, iou_threshold) else "Empty", axis=1)

    # Create a copy of the image for visualization
    marked_img = img.copy()
    for _, chair in chairs_df.iterrows():
        x1, y1, x2, y2 = int(chair["x1"]), int(chair["y1"]), int(chair["x2"]), int(chair["y2"])
        center_x, center_y = int(chair["center_x"]), int(chair["center_y"])
        radius = int((x2 - x1) / 2)  # Approximate radius for the circle
        if chair["Status"] == "Empty":
            # Circle for empty chairs (green)
            cv2.circle(marked_img, (center_x, center_y), radius, (0, 255, 0), 2)
        else:
            # Rectangle for occupied chairs (red)
            cv2.rectangle(marked_img, (x1, y1), (x2, y2), (0, 0, 255), 2)
        # Label the chair with Seat ID and Status
        cv2.putText(marked_img, f"{chair['Seat ID']} ({chair['Status']})", (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0) if chair["Status"] == "Empty" else (0, 0, 255), 2)

    chairs_df.to_excel(f"../data/output/seat_status_{os.path.basename(image_path).split('.')[0]}.xlsx", index=False)
    print(f"Initial Seat Status for {os.path.basename(image_path)}:\n", chairs_df[["Seat ID", "Status"]])
    return chairs_df, img, marked_img

In [5]:
# Step 4: Define Seat Allocation Function
def allocate_seats(chairs_df, num_students, image_name):
    chairs_df = chairs_df.copy()
    assignments = []

    # Find the first occupied seat (if any) to start clustering
    occupied_seats = chairs_df[chairs_df["Status"] == "Occupied"]
    if not occupied_seats.empty:
        last_assigned_index = occupied_seats.index[0]
    else:
        empty_seats = chairs_df[chairs_df["Status"] == "Empty"]
        if empty_seats.empty:
            return "No empty seats available.", chairs_df
        last_assigned_index = empty_seats.index[0]
        chairs_df.loc[last_assigned_index, "Status"] = "Occupied"
        assignments.append(f"Student 1 → {chairs_df.loc[last_assigned_index, 'Seat ID']}")
        num_students -= 1

    # Allocate remaining students to the next available seats in the sorted order
    for i in range(num_students):
        next_index = last_assigned_index + 1
        while next_index < len(chairs_df) and chairs_df.loc[next_index, "Status"] != "Empty":
            next_index += 1
        
        if next_index >= len(chairs_df):
            next_index = chairs_df[chairs_df["Status"] == "Empty"].index[0] if not chairs_df[chairs_df["Status"] == "Empty"].empty else None
            if next_index is None:
                return assignments + ["No more empty seats available."], chairs_df

        chairs_df.loc[next_index, "Status"] = "Occupied"
        assignments.append(f"Student {i+2} → {chairs_df.loc[next_index, 'Seat ID']}")
        last_assigned_index = next_index

    chairs_df.to_excel(f"../data/output/seat_status_updated_{image_name.split('.')[0]}.xlsx", index=False)
    return assignments, chairs_df

In [6]:
# Step 5: Define Visualization Function
def visualize_results(original_img, marked_img, chairs_df, assignments, image_name, save_visualization=False):
    # Convert BGR (OpenCV) to RGB (Matplotlib)
    original_img_rgb = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
    marked_img_rgb = cv2.cvtColor(marked_img, cv2.COLOR_BGR2RGB)

    # Create a figure with two subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

    # Plot the original image
    ax1.imshow(original_img_rgb)
    ax1.set_title(f"Original Image: {image_name}")
    ax1.axis("off")

    # Plot the marked image
    ax2.imshow(marked_img_rgb)
    ax2.set_title(f"Marked Image: {image_name} (Empty: Green Circles, Occupied: Red Rectangles)")
    ax2.axis("off")

    if save_visualization:
        plt.savefig(f"../data/output/visualization_{image_name.split('.')[0]}.png", bbox_inches="tight")
    plt.show()

    # Print seat assignments
    print(f"Seat Assignments for {image_name}:\n", "\n".join(assignments))
    print(f"Updated Seat Status for {image_name}:\n", chairs_df[["Seat ID", "Status"]])

In [7]:
# Step 6: Process All Images in a Loop
def main(num_students=3, iou_threshold=0.5, save_visualization=False):
    # Define the images directory (relative path from src/ to root)
    images_dir = "../data/images"

    # Check if the directory exists
    if not os.path.exists(images_dir):
        raise FileNotFoundError(f"Directory {images_dir} does not exist. Please check the path.")

    # Get all image files in the directory
    image_files = [f for f in os.listdir(images_dir) if f.endswith(('.jpeg', '.jpg', '.png'))]
    if not image_files:
        print("No images found in ../data/images/ directory.")
        return

    for image_name in image_files:
        print(f"\nProcessing {image_name}...\n" + "="*50)

        # Construct the image path
        image_path = os.path.join(images_dir, image_name)

        # Step 1: Detect chairs and determine occupancy
        chairs_df, original_img, marked_img = detect_chairs(image_path, iou_threshold)

        # Step 2: Allocate seats
        assignments, updated_chairs_df = allocate_seats(chairs_df, num_students, image_name)

        # Step 3: Visualize results
        visualize_results(original_img, marked_img, updated_chairs_df, assignments, image_name, save_visualization)

# Run the main loop
num_students = 3  # Number of students to allocate seats to
iou_threshold = 0.5  # IoU threshold for determining occupancy
save_visualization = True  # Whether to save visualizations to disk
main(num_students=num_students, iou_threshold=iou_threshold, save_visualization=save_visualization)


Processing classroom_image7.jpeg...


Using cache found in /home/shreyx/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-3-24 Python-3.12.2 torch-2.3.1 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Initial Seat Status for classroom_image7.jpeg:
     Seat ID Status
0    Seat 1  Empty
1    Seat 2  Empty
2    Seat 3  Empty
3    Seat 4  Empty
4    Seat 5  Empty
5    Seat 6  Empty
6    Seat 7  Empty
7    Seat 8  Empty
8    Seat 9  Empty
9   Seat 10  Empty
10  Seat 11  Empty
11  Seat 12  Empty
12  Seat 13  Empty
13  Seat 14  Empty
14  Seat 15  Empty
15  Seat 16  Empty
16  Seat 17  Empty
17  Seat 18  Empty
18  Seat 19  Empty
19  Seat 20  Empty
20  Seat 21  Empty
21  Seat 22  Empty
22  Seat 23  Empty
23  Seat 24  Empty
24  Seat 25  Empty


Using cache found in /home/shreyx/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-3-24 Python-3.12.2 torch-2.3.1 CPU



Seat Assignments for classroom_image7.jpeg:
 Student 1 → Seat 1
Student 2 → Seat 2
Student 3 → Seat 3
Updated Seat Status for classroom_image7.jpeg:
     Seat ID    Status
0    Seat 1  Occupied
1    Seat 2  Occupied
2    Seat 3  Occupied
3    Seat 4     Empty
4    Seat 5     Empty
5    Seat 6     Empty
6    Seat 7     Empty
7    Seat 8     Empty
8    Seat 9     Empty
9   Seat 10     Empty
10  Seat 11     Empty
11  Seat 12     Empty
12  Seat 13     Empty
13  Seat 14     Empty
14  Seat 15     Empty
15  Seat 16     Empty
16  Seat 17     Empty
17  Seat 18     Empty
18  Seat 19     Empty
19  Seat 20     Empty
20  Seat 21     Empty
21  Seat 22     Empty
22  Seat 23     Empty
23  Seat 24     Empty
24  Seat 25     Empty

Processing classroom_image11.jpeg...


...

Initial Seat Status for classroom_image4.jpeg:
     Seat ID Status
0    Seat 1  Empty
1    Seat 2  Empty
2    Seat 3  Empty
3    Seat 4  Empty
4    Seat 5  Empty
5    Seat 6  Empty
6    Seat 7  Empty
7    Seat 8  Empty
8    Seat 9  Empty
9   Seat 10  Empty
10  Seat 11  Empty
11  Seat 12  Empty
12  Seat 13  Empty
13  Seat 14  Empty
14  Seat 15  Empty
15  Seat 16  Empty


Using cache found in /home/shreyx/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-3-24 Python-3.12.2 torch-2.3.1 CPU



Seat Assignments for classroom_image4.jpeg:
 Student 1 → Seat 1
Student 2 → Seat 2
Student 3 → Seat 3
Updated Seat Status for classroom_image4.jpeg:
     Seat ID    Status
0    Seat 1  Occupied
1    Seat 2  Occupied
2    Seat 3  Occupied
3    Seat 4     Empty
4    Seat 5     Empty
5    Seat 6     Empty
6    Seat 7     Empty
7    Seat 8     Empty
8    Seat 9     Empty
9   Seat 10     Empty
10  Seat 11     Empty
11  Seat 12     Empty
12  Seat 13     Empty
13  Seat 14     Empty
14  Seat 15     Empty
15  Seat 16     Empty

Processing classroom_image12.jpeg...


Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Initial Seat Status for classroom_image12.jpeg:
     Seat ID Status
0    Seat 1  Empty
1    Seat 2  Empty
2    Seat 3  Empty
3    Seat 4  Empty
4    Seat 5  Empty
5    Seat 6  Empty
6    Seat 7  Empty
7    Seat 8  Empty
8    Seat 9  Empty
9   Seat 10  Empty
10  Seat 11  Empty
11  Seat 12  Empty
12  Seat 13  Empty
13  Seat 14  Empty
14  Seat 15  Empty
15  Seat 16  Empty
16  Seat 17  Empty
17  Seat 18  Empty
18  Seat 19  Empty
19  Seat 20  Empty
20  Seat 21  Empty
21  Seat 22  Empty
22  Seat 23  Empty
23  Seat 24  Empty
24  Seat 25  Empty
Seat Assignments for classroom_image12.jpeg:
 Student 1 → Seat 1
Student 2 → Seat 2
Student 3 → Seat 3
Updated Seat Status for classroom_image12.jpeg:
     Seat ID    Status
0    Seat 1  Occupied
1    Seat 2  Occupied
2    Seat 3  Occupied
3    Seat 4     Empty
4    Seat 5     Empty
5    Seat 6     Empty
6    Seat 7     Empty
7    Seat 8     Empty
8    Seat 9     Empty
9   Seat 10     Empty
10  Seat 11     Empty
11  Seat 12     Empty
12  Seat 13     Emp

In [8]:
# Step 7: Display a Test Image with Empty Chairs Circled
def display_test_image(image_name, iou_threshold=0.5):
    # Construct the image path
    image_path = os.path.join("../data/images", image_name)

    # Detect chairs and get the marked image
    chairs_df, original_img, marked_img = detect_chairs(image_path, iou_threshold)

    # Visualize the original and marked images
    original_img_rgb = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
    marked_img_rgb = cv2.cvtColor(marked_img, cv2.COLOR_BGR2RGB)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
    ax1.imshow(original_img_rgb)
    ax1.set_title(f"Original Test Image: {image_name}")
    ax1.axis("off")
    ax2.imshow(marked_img_rgb)
    ax2.set_title(f"Test Image: {image_name} (Empty: Green Circles, Occupied: Red Rectangles)")
    ax2.axis("off")
    plt.show()

# Display a test image
test_image_name = "classroom_image1.jpeg"
display_test_image(test_image_name, iou_threshold=0.5)

Using cache found in /home/shreyx/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-3-24 Python-3.12.2 torch-2.3.1 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Initial Seat Status for classroom_image1.jpeg:
     Seat ID Status
0    Seat 1  Empty
1    Seat 2  Empty
2    Seat 3  Empty
3    Seat 4  Empty
4    Seat 5  Empty
5    Seat 6  Empty
6    Seat 7  Empty
7    Seat 8  Empty
8    Seat 9  Empty
9   Seat 10  Empty
10  Seat 11  Empty
11  Seat 12  Empty
